In [37]:
baseline = {
    "ntrain": ["4"],
    "valid_frac": ["1"],
    "batch_size": ["256"],
    "epochs": ["18"],
    "loss": ["SoftMaskWeightedMeanSquaredErrorWithTotal"],
    "soft_mask": ["True"],    
    "disc_do_total": ["True"],
    
    "loss_weight": ["1.0"], # lambda
    "loss_tau":  ["1e-05"], # tau 
    "gan_total_weight":  ["4.5"], # mu
    "lr_decay_disc":  ["6e-08"],
    "lr_decay_gen":  ["6e-08"],
    
    "noise_dim": ["0"],

    "nfilters": ["8"],
    "gen_filter_size" : [ "3" ],
#     "encoding_filter_size": 3,
#     "decoding_filter_size": 3,
    "disc_nfilters": [ "4" ],
    "disc_total_layers": [ "64,64,32,16" ],
}

scan_1 = {
    
    "loss_weight": [ "1.0", "0.8", "1.2" ],
    "loss_tau": [ "1e-05", "8e-6", "1.2e-5" ],
    "gan_total_weight": [ "4.5", "5.0", "5.5" ],
    "lr_decay_disc": [ "6e-08", "1e-7", "4e-8" ], 
    "lr_decay_gen":  [ "6e-08", "1e-7", "4e-8" ]
} ## 243


scan_2 = {
    "noise_dim": [ "0", "5", "10" ],
    "nfilters": [ "8", "6", "10" ],
    "gen_filter_size": [ "3", "5" ],   
#     "encoding_filter_size": [ 3, 5 ],
#     "decoding_filter_size": [ 3, 5 ],
    "disc_nfilters": [ "4", "5", "6"],
    "disc_total_layers": [ "64,64,32,16", "64,32,16,8" ]
} ## 324


In [38]:
from copy import copy

In [39]:
grid_1 = copy(baseline)
grid_1.update(scan_1)

grid_2 = copy(baseline)
grid_2.update(scan_2)


In [40]:
grid_1

{'batch_size': ['256'],
 'disc_do_total': ['True'],
 'disc_nfilters': ['4'],
 'disc_total_layers': ['64,64,32,16'],
 'epochs': ['18'],
 'gan_total_weight': ['4.5', '5.0', '5.5'],
 'gen_filter_size': ['3'],
 'loss': ['SoftMaskWeightedMeanSquaredErrorWithTotal'],
 'loss_tau': ['1e-05', '8e-6', '1.2e-5'],
 'loss_weight': ['1.0', '0.8', '1.2'],
 'lr_decay_disc': ['6e-08', '1e-7', '4e-8'],
 'lr_decay_gen': ['6e-08', '1e-7', '4e-8'],
 'nfilters': ['8'],
 'noise_dim': ['0'],
 'ntrain': ['4'],
 'soft_mask': ['True'],
 'valid_frac': ['1']}

In [41]:
grid_2

{'batch_size': ['256'],
 'disc_do_total': ['True'],
 'disc_nfilters': ['4', '5', '6'],
 'disc_total_layers': ['64,64,32,16', '64,32,16,8'],
 'epochs': ['18'],
 'gan_total_weight': ['4.5'],
 'gen_filter_size': ['3', '5'],
 'loss': ['SoftMaskWeightedMeanSquaredErrorWithTotal'],
 'loss_tau': ['1e-05'],
 'loss_weight': ['1.0'],
 'lr_decay_disc': ['6e-08'],
 'lr_decay_gen': ['6e-08'],
 'nfilters': ['8', '6', '10'],
 'noise_dim': ['0', '5', '10'],
 'ntrain': ['4'],
 'soft_mask': ['True'],
 'valid_frac': ['1']}

In [42]:
from sklearn.model_selection import ParameterSampler

In [43]:
sampler_1 = ParameterSampler(grid_1, 25)
sampler_2 = ParameterSampler(grid_2, 25)


In [44]:
def mk_cmdline(params):
    
    gen_filter_size = params.pop("gen_filter_size")
    params["encoding_filter_size"] = gen_filter_size
    params["decoding_filter_size"] = gen_filter_size
    
    return " ".join( [ "--Parameters.%s=%s" % (x[0],x[1]) for x in params.items() ]  )
    

In [45]:
from random import shuffle

cmdlines = [ mk_cmdline(x) for x in sampler_1 ] + [ mk_cmdline(x) for x in sampler_2 ] 
shuffle(cmdlines)


In [49]:
with open("submit_batch.sh","w+") as fout:
    for ijob,cmdline in enumerate(cmdlines):
        fout.write("sbatch my_job.sh -J scan_job%d python train_gan.py --Parameters.monitor_dir=log/scan_job%d %s\n" % (ijob,ijob,cmdline) )
    fout.close()